In [1]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

Preprocessing :
Using wavelet transform as a feature for training our model

In a wavelet transformed image you can see edges clearly and that can give us clues on various facial features such as eyes, nose, lips

https://medium.com/@koushikc2000/2d-discrete-wavelet-transformation-and-its-applications-in-digital-image-processing-using-matlab-1f5c68672de3